# Process Sentinel-2 imagery for plotting
This script makes RGB true-color images from Sentinel-2 data from Jan 2 2019 for the area of this study, and creates smaller images for each of the 4 melt ponds. 

In [1]:
%matplotlib notebook
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio import plot
from rasterio import warp
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.enums import ColorInterp
from rasterio.windows import Window
from rasterio.transform import TransformMethodsMixin
from scipy import interpolate
from os import listdir, makedirs
from os.path import isfile, join, exists
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pickle as pkl

## To get the imagery, copy from S3 
...because it's in the Long Term Archive at ESA

The S3 locations can be found through this tool: https://apps.sentinel-hub.com/eo-browser/?zoom=5&lat=-71.62214&lng=62.24639&themeId=DEFAULT-THEME

(need AWS account because it's in a requester pays bucket)

In [7]:
s2dat = 'data/sentinel2/'
if not exists(s2dat):
    makedirs(s2dat)
for tilenum in np.arange(8,14):
    thisdir = s2dat + 'raw/%d/' % tilenum
    if not exists(thisdir):
        makedirs(thisdir)
        
# # these are the commands for downloading S2 tiles from AWS S3 using AWS CLI
# !aws s3 cp s3://sentinel-s2-l2a/tiles/41/C/NU/2019/1/2/0/R10m/ /mnt/c/Users/phili/Google\ Drive/Documents/SIO/PycharmProjects/ameryMeltICESat2/data/sentinel2/raw/8/ --recursive --request-payer
# !aws s3 cp s3://sentinel-s2-l2a/tiles/42/C/VD/2019/1/2/0/R10m/ /mnt/c/Users/phili/Google\ Drive/Documents/SIO/PycharmProjects/ameryMeltICESat2/data/sentinel2/raw/9/ --recursive --request-payer
# !aws s3 cp s3://sentinel-s2-l2a/tiles/41/D/NA/2019/1/2/0/R10m/ /mnt/c/Users/phili/Google\ Drive/Documents/SIO/PycharmProjects/ameryMeltICESat2/data/sentinel2/raw/10/ --recursive --request-payer
# !aws s3 cp s3://sentinel-s2-l2a/tiles/41/C/NV/2019/1/2/0/R10m/ /mnt/c/Users/phili/Google\ Drive/Documents/SIO/PycharmProjects/ameryMeltICESat2/data/sentinel2/raw/11/ --recursive --request-payer
# !aws s3 cp s3://sentinel-s2-l2a/tiles/41/D/PA/2019/1/2/0/R10m/ /mnt/c/Users/phili/Google\ Drive/Documents/SIO/PycharmProjects/ameryMeltICESat2/data/sentinel2/raw/12/ --recursive --request-payer
# !aws s3 cp s3://sentinel-s2-l2a/tiles/41/C/PV/2019/1/2/0/R10m/ /mnt/c/Users/phili/Google\ Drive/Documents/SIO/PycharmProjects/ameryMeltICESat2/data/sentinel2/raw/13/ --recursive --request-payer

## make 8-bit RGB true color images

In [3]:
%%time
tc = s2dat + 'truecolor/'
if not exists(tc):
    makedirs(tc)
    
for tilenum in np.arange(8,14):
    imgdat = s2dat + 'raw/%d/' % tilenum
    r = rio.open(imgdat+'B04.jp2', driver='JP2OpenJPEG')
    g = rio.open(imgdat+'B03.jp2', driver='JP2OpenJPEG')
    b = rio.open(imgdat+'B02.jp2', driver='JP2OpenJPEG')

    #export true color image
    outname = tc + 'SentinelTrueColor%d.tiff' % tilenum
    if tilenum == 9:  # this image is a different CRS so save as different filename
        outname = tc + 'SentinelTrueColor9_alt.tiff'
    trueColor = rio.open(outname, 'w', driver='Gtiff',
                         width=r.width, height=r.height,
                         count=3,
                         crs=r.crs,
                         transform=r.transform,
                         dtype='uint8')
    trueColor.profile['photometric'] = "RGB"
    trueColor.colorinterp = [ColorInterp.red, ColorInterp.green, ColorInterp.blue]
    
    factor = 0.02
    rband = r.read(1)*factor
    gband = g.read(1)*factor
    bband = b.read(1)*factor
    rband[rband>255] = 255
    gband[gband>255] = 255
    bband[bband>255] = 255
    
    # write to file (need to multiply by a factor to go from uint16 to uint8)
    trueColor.write(rband.astype(np.uint8), 1)
    trueColor.write(gband.astype(np.uint8), 2)
    trueColor.write(bband.astype(np.uint8), 3)
    trueColor.close()

CPU times: user 15min 15s, sys: 1min, total: 16min 16s
Wall time: 3min 17s


## write smaller, windowed files for individual ponds 
(these are used for the inset axes where the ponds are shown magnified)

In [4]:
%%time
ponddir = s2dat + 'ponds/'
if not exists(ponddir):
    makedirs(ponddir)

# limits for images in CRS
xls = [(634423.1752651753, 643701.4824069294),
       (635265.0812945361, 648049.5670659576),
       (659819.6180634786, 669507.1880767047),
       (665429.7291041224, 675379.7684254167)]
yls = [(1891121.2487189926, 1900283.3348944616),
       (1900696.8178272631, 1913898.1890042746),
       (2014352.220285779, 2024130.328336512),
       (2039552.1048050749, 2049167.6890231324)]

tiles = [13,13,12,12]
for pid, tilenum in enumerate(tiles):
    pond = 'pond%d' % (pid+1)
    imgdat = s2dat + 'raw/%d/' % tilenum
    
    # make window
    bd = rio.open(imgdat+'B04.jp2', driver='JP2OpenJPEG')
    ll = TransformMethodsMixin.index(bd,xls[pid][0],yls[pid][0])
    ur = TransformMethodsMixin.index(bd,xls[pid][1],yls[pid][1])
    col_off = np.min((ll[1],ur[1]))
    row_off = np.min((ll[0],ur[0]))
    width = np.abs(ll[1]-ur[1])
    height = np.abs(ll[0]-ur[0])
    wd = Window(col_off=col_off, row_off=row_off, width=width, height=width)

    # set up true color image for export
    outname = ponddir + '%s_alt.tiff' % pond
    trueColor = rio.open(outname, 'w', driver='Gtiff',
                         width=width, height=width,
                         count=3,
                         crs=bd.crs,
                         #transform=bd.transform,
                         transform = bd.window_transform(wd),
                         dtype='uint8')
    trueColor.profile['photometric'] = "RGB"
    trueColor.colorinterp = [ColorInterp.red, ColorInterp.green, ColorInterp.blue]
    
    # read in windowed bands
    r = rio.open(imgdat+'B04.jp2', driver='JP2OpenJPEG')
    g = rio.open(imgdat+'B03.jp2', driver='JP2OpenJPEG')
    b = rio.open(imgdat+'B02.jp2', driver='JP2OpenJPEG')
    factor = 0.025
    rband = r.read(1,window=wd)*factor
    gband = g.read(1,window=wd)*factor
    bband = b.read(1,window=wd)*factor
    rband[rband>255] = 255
    gband[gband>255] = 255
    bband[bband>255] = 255
    
    # write to file (need to multiply by a factor to go from uint16 to uint8)
    trueColor.write(rband.astype(np.uint8), 1)
    trueColor.write(gband.astype(np.uint8), 2)
    trueColor.write(bband.astype(np.uint8), 3)
    trueColor.close()

## reproject lower right image
it's in a different CRS, so need to reproject to plot together with the others on the map

In [5]:
%%time
fnsrc = tc + 'SentinelTrueColor9_alt.tiff'
fndst = tc + 'SentinelTrueColor9.tiff'

refImg = rio.open('data/sentinel2/truecolor/SentinelTrueColor13.tiff')
dst_crs = refImg.crs

with rio.open(fnsrc) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rio.open(fndst, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

## scaled down true color images
(for figure 1, where the whole scene is shown)

In [6]:
%%time
for tilenum in np.arange(8,14):
    imgdat = tc + 'SentinelTrueColor%d.tiff' % tilenum
    outname = tc + 'SentinelTrueColor%d_small.tiff' % tilenum
    
    upscale_factor = 0.2
    refImg = rio.open(imgdat)
    h = int(refImg.height * upscale_factor)
    w = int(refImg.width * upscale_factor)
    data = refImg.read(out_shape=(3,h,w),resampling=Resampling.average)

    # scale image transform
    transform = refImg.transform * refImg.transform.scale((refImg.width / data.shape[-1]),(refImg.height / data.shape[-2]))
    trueColor = rio.open(outname, 'w', driver='Gtiff',
                         width=w, height=h,
                         count=3,
                         crs=refImg.crs,
                         transform=transform,
                         dtype='uint8')
    trueColor.profile['photometric'] = "RGB"
    trueColor.colorinterp = [ColorInterp.red, ColorInterp.green, ColorInterp.blue]
    
    # write to file (need to multiply by a factor to go from uint16 to uint8)
    trueColor.write(data.astype(np.uint8), [1,2,3])
    trueColor.close()